# Lab 1: Descent (Part 2)

Digging at the lowest point of Olduvai Gorge, you discover a hatch:

![Hatch](./img/trapdoor.png)

You descend a ladder until you reach a sloping surface. Armed only with a GPS and torch (both the light source and the tensor manipulation package), you can't see much beyond your own two feet. However, you can determine your current position and the current gradient, as well as whether you see an exit from your current position.

In [ ]:
from levels import Environment, Level1
level1 = Level1()
position = level1.current_position()
print(f"Current position:     {level1.current_position()}")
print(f"Current gradient:     {level1.gradient(position)}")
print(f"Is the exit visible?  {level1.can_see_exit(position)}")

You can also step to a new position and check that out.

In [ ]:
from torch import tensor
level1.step_to(tensor([3.0, 4.0]))
new_position = level1.current_position()
print(f"Current position:     {new_position}")
print(f"Current gradient:     {level1.gradient(new_position)}")
print(f"Is the exit visible?  {level1.can_see_exit(new_position)}")

You're a little frightened of the dark, so you decide that you won't take more than 100 steps before retreating back to the entry hatch and trying a new strategy. This is tracked by the environment, which returns the status ```EXCEEDED_STEP_LIMIT``` if you've taken too many steps.

In [ ]:
level1 = Level1()
position = level1.current_position()
for i in range(101):
    position = position + tensor([0.01, 0.01])
    level1.step_to(position)    
    if level1.status() == Environment.ACTIVELY_SEARCHING:
        status = "ACTIVELY SEARCHING"
    elif level1.status() == Environment.EXCEEDED_STEP_LIMIT:
        status = "EXCEEDED STEP LIMIT"
    elif level1.status() == Environment.FOUND_EXIT:
        status = "FOUND EXIT"
    print(f"{status} at {level1.current_position()}")

Here is the visualization of a journey that begins at point (0,0), then proceeds to points (3,1), (6,-1), and (5,-2), without finding the exit. The starting point is in orange, the intermediate points are in gray, and the final point is either red (if no exit has been found) or green (if an exit has been found).

In [ ]:
%matplotlib inline
from torch import tensor
from levels import visualize_journey
positions = [tensor([0.0, 0.0]), tensor([3.0, 1.0]), 
             tensor([6.0, -1.0]), tensor([5.0, -2.0])]
visualize_journey(positions, False)

Luckily, you've studied gradient descent algorithms, so you know what to do.

----
### Question 1
----

Implement a general-purpose gradient descent algorithm, so that you can start exploring the environment. Specifically, complete the function ```grad_descent``` in ```algorithms.py```, according to the specifications in its comments.

There's a unit test in test.py, so that you can check your algorithm is working properly. Run it from the command line as follows:

    python -m unittest test.Q1    
    
Once it's working, you should be ready to find the lowest point in the environment.

In [ ]:
from algorithms import vanilla_grad_descent
from levels import Level1, visualize_journey
env = Level1()
points = vanilla_grad_descent(0.95, env)
if len(points[1:]) == 51:
    visualize_journey(points, env.can_see_exit(points[-1]))
else:
    print("Looks like there's still a bug in your grad_descent code.")

After a mere 51 steps, you catch sight of yet another hatch:

![Hatch](./img/trapdoor2.png)

You take another ladder down into yet another dark environment. Well, it worked last time, so let's try vanilla gradient descent once again.

In [ ]:
from levels import Level2
env = Level2()
points = vanilla_grad_descent(0.95, env)
visualize_journey(points, env.can_see_exit(points[-1]))

After 100 steps, you still haven't found the exit, so you retrace your steps back to the entrance. Visualizing your previous journey, it seems like you're in a narrow ravine with steeply sloping sides, which is causing you to go back and forth sharply across the width of the ravine, but not make much forward progress along its length. You've read about this phenomenon in class, and hypothesize that one way to deal with it is to use gradient descent with *momentum*. 

----
### Question 2
----

Complete the implementation of gradient descent with momentum (```momentum_grad_descent```) in ```algorithms.py```, according to the specifications in its comments.

Again we've provided a unit test in test.py, so that you can check your algorithm is working properly. Run it from the command line as follows:

    python -m unittest test.GradientDescentTests.test_momentum
    
Confident in your test-passing implementation, you steel yourself and head into the darkness.

In [ ]:
from algorithms import momentum_grad_descent
from levels import Level2, visualize_journey
env = Level2()
points = momentum_grad_descent(0.95, env)
if len(points[1:]) == 93:
    visualize_journey(points, env.can_see_exit(points[-1]))
else:
    print("Looks like there's still a bug in your grad_descent code.")

After 93 steps, you find a third hatch:

![Hatch](./img/trapdoor3.png)

Again, you go down. You try a few steps of gradient descent with momentum...

In [ ]:
from algorithms import momentum_grad_descent
from levels import Level3
env = Level3()
points = momentum_grad_descent(0.95, env)
visualize_journey(points, env.can_see_exit(points[-1]))

...but quickly realize you're spinning out of control (notice the scale of the axes). It seems that your step size is too large. You go back to vanilla gradient descent with a smaller step size.

In [ ]:
from algorithms import momentum_grad_descent
from levels import Level3
env = Level3()
points = vanilla_grad_descent(0.05, env)
visualize_journey(points, env.can_see_exit(points[-1]))

Unfortunately, you find another hatch. Hopefully this the last one.

![Hatch](./img/trapdoor4.png)

You try out vanilla gradient descent with the same conservative step size.


In [ ]:
from algorithms import momentum_grad_descent
from levels import Level4
env = Level4()
points = vanilla_grad_descent(0.05, env)
visualize_journey(points, env.can_see_exit(points[-1]))

After 100 steps, you declare defeat and go back to the entrance. You're a bit tired of guessing the correct step size, so you'd like to try a gradient descent method with an adaptive step size (i.e. one where the step size is a function of time). You can't help but notice that the slopes of this level aren't particularly smooth. You proceeded directly east for a long time before the gradient started leading you south. Therefore you don't want to use an adaptive method where the step size decreases in all dimensions at the same rate, because by the time you reach the south-leading slope, your step size would likely be too tiny to make any good progress.

Luckily, you remember AdaGrad is a gradient descent method where the rate changes independently in each dimension.

----
### Question 3
----

Complete the implementation of AdaGrad (```adagrad```) in ```algorithms.py```, according to the specifications in its comments.

We've provided a unit test in test.py, so that you can check your algorithm is working properly. Run it from the command line as follows:

    python -m unittest test.GradientDescentTests.test_adagrad
    
Confident in your implementation, you try navigating the fourth level again.

In [ ]:
from algorithms import adagrad
from levels import Level4, visualize_journey
env = Level4()
points = adagrad(0.95, env)
if len(points[1:]) == 101:
    visualize_journey(points, env.can_see_exit(points[-1]))
else:
    print("Looks like there's still a bug in your adagrad code.")

Seems like you're close, but no cigar. You seem to remember learning that AdaGrad can sometimes converge too quickly, and that RmsProp might be a better choice for some topologies. Looks like it's back to implementation!

----
### Question 4
----

Complete the implementation of RmsProp (```rmsprop```) in ```algorithms.py```, according to the specifications in its comments.

We've provided a unit test in test.py, so that you can check your algorithm is working properly. Run it from the command line as follows:

    python -m unittest test.GradientDescentTests.test_rmsprop
    
Third time's the charm! You head out again, into the darkness.

In [ ]:
from algorithms import rmsprop
from levels import Level4, visualize_journey
env = Level4()
points = rmsprop(0.4, 0.95, env)
if len(points[1:]) == 41:
    visualize_journey(points, env.can_see_exit(points[-1]))
else:
    print("Looks like there's still a bug in your rmsprop code.")

This time, there's no hatch, just a mysterious chest.

![Chest](./img/chest.png)

You open it. Inside rests the resplendent Chekhov's Sun. What will you do with the world's largest yellow diamond? After correctly implementing four different gradient descent algorithms, the answer becomes clear.

In [ ]:
from levels import LevelX
from algorithms import vanilla_grad_descent, rmsprop
from algorithms import momentum_grad_descent, adagrad

def answer():
    points = vanilla_grad_descent(0.95, LevelX())
    points2 = momentum_grad_descent(0.95, LevelX())
    points3 = adagrad(0.95, LevelX())
    points4 = rmsprop(0.4, 0.95, LevelX())
    ALPHABET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    letter1 = ALPHABET[int(points[3][0].item()) - 11]
    letter2 = ALPHABET[int(points2[3][0].item()) - 8]
    letter3 = ALPHABET[int(points3[3][0].item()) + 16]
    letter4 = ALPHABET[int(points4[3][0].item()) + 3]
    letter5 = ALPHABET[int(points2[5][0].item()) - 20]
    letter6 = ALPHABET[int(points4[5][0].item()) + 11]
    word = '{}{}{}{}{}{}'.format(letter1, letter2, letter3, letter4, letter5, letter6)
    return word

def submit(response):
    import rpyc
    c = rpyc.connect("137.165.10.56", 18861)
    return c.root.submit_response('q2', response)

print('You submit the password {} to the server.'.format(answer()))
submit(answer())